In [1]:
pip install torch torchvision torchaudio
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121

SyntaxError: invalid syntax (1356884835.py, line 1)

In [2]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.eval()


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [3]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)
    

In [12]:
# sample execution (requires torchvision)

filename = "q.jpg"

from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([-2.4770e+00, -4.8352e+00, -1.4745e+00, -2.2314e+00, -2.5574e+00,
         5.6773e-01,  3.7111e+00, -2.8427e+00,  9.3308e-02, -8.0384e-01,
         1.1397e+00, -2.1540e+00, -2.0175e+00, -2.7175e+00, -2.5519e+00,
         1.1289e+00, -3.9042e+00,  5.2653e-01, -2.8923e-01, -3.5874e+00,
        -2.0677e+00, -2.7765e+00, -5.5608e+00, -4.6808e+00,  4.4263e-01,
        -3.8819e-01, -1.2223e+00, -1.5925e+00,  9.9867e-02, -1.4305e+00,
        -3.5502e+00, -3.7989e+00, -2.6157e+00, -1.6383e+00, -3.4433e-01,
        -9.2153e-01,  3.0364e+00, -8.1369e-01, -4.8073e+00, -3.6462e-01,
        -5.4400e+00, -4.3037e+00, -5.4703e+00, -2.5045e+00, -4.4030e+00,
        -5.3908e-01, -6.8342e+00, -3.7445e+00, -2.1431e+00, -4.6500e-01,
        -9.0628e-01,  3.3689e+00, -4.4622e+00, -2.9843e+00, -1.7956e+00,
        -3.1368e+00, -4.4531e+00, -1.5148e+00, -4.5795e+00, -4.3974e+00,
        -2.0946e+00, -1.9760e-01,  5.1472e-01, -3.1728e+00, -3.8567e+00,
        -2.1516e+00, -1.3802e+00, -3.5528e+00, -3.6

In [5]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2024-03-09 20:43:04--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.2’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.001s  

2024-03-09 20:43:04 (7.14 MB/s) - ‘imagenet_classes.txt.2’ saved [10472/10472]



In [13]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 10)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())
    

German shepherd 0.3875349164009094
malinois 0.31557729840278625
muzzle 0.07319677621126175
kelpie 0.06898795813322067
Border terrier 0.03275055065751076
wallaby 0.026657309383153915
bluetick 0.014524184167385101
Rottweiler 0.013454802334308624
Norwegian elkhound 0.01168107707053423
miniature pinscher 0.009507712908089161
